今日の天気をprint

In [5]:
import requests
from bs4 import BeautifulSoup

def get_weather():
    url = "https://tenki.jp/forecast/3/17/4610/14103/"
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTPステータスコードを確認：200が成功
        response.encoding = response.apparent_encoding #文字化けしないようエンコード

        soup = BeautifulSoup(response.content, "html.parser")

        weather_element = soup.find("p", class_="weather-telop")  # クラスで選択

        if weather_element:
            today_weather = weather_element.text.strip()
            return today_weather
        else:
            return "Could not find weather information."

    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {e}"

if __name__ == "__main__":
    weather = get_weather()

weather


'晴一時雨'

10日間天気をcsv出力

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_weather_forecast():
    url = "https://tenki.jp/forecast/3/17/4610/14103/"
    try:
        response = requests.get(url)
        response.raise_for_status()
        response.encoding = response.apparent_encoding

        soup = BeautifulSoup(response.content, "html.parser")

        forecast_table = soup.find("table", class_="forecast-point-week forecast-days-long")

        forecast_data = {"日付": [], "天気": [], "最高気温": [], "最低気温":[],}

        if forecast_table:
            rows = forecast_table.find_all("tr")

            # 日付行データを格納
            date_row = rows[0]
            tds = date_row.find_all("td")
            for td in tds:
              date = td.find("p", class_="date-box").get_text(strip=True)
              forecast_data["日付"].append(date)

            # 天気行データを格納
            date_row = rows[1]
            tds = date_row.find_all("td")
            for td in tds:
              weather = td.find("p").get_text(strip=True)
              forecast_data["天気"].append(weather)

            # 気温行データを格納
            date_row = rows[2]
            tds = date_row.find_all("td")
            for td in tds:
              high_temp = td.find("p", class_="high-temp").get_text(strip=True)
              low_temp = td.find("p", class_="low-temp").get_text(strip=True)
              forecast_data["最高気温"].append(high_temp)
              forecast_data["最低気温"].append(low_temp)

            return forecast_data
        else:
            return "Could not find weather forecast table."

    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {e}"

forecast = get_weather_forecast()
df = pd.DataFrame(forecast)
# print(df.to_string(index=False))
df.to_csv("forecast.csv", index=False, encoding="utf-8-sig")